In [91]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV

from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV

train = pd.read_csv("train_kOBLwZA.csv")
test=pd.read_csv("test_t02dQwI.csv")
sample=pd.read_csv("SampleSubmission_TmnO39y.txt")

In [92]:
train.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

In [93]:
df=pd.concat([train,test],axis="index")

C:\data\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [94]:
df.shape

(14204, 12)

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14204 entries, 0 to 5680
Data columns (total 12 columns):
Item_Fat_Content             14204 non-null object
Item_Identifier              14204 non-null object
Item_MRP                     14204 non-null float64
Item_Outlet_Sales            8523 non-null float64
Item_Type                    14204 non-null object
Item_Visibility              14204 non-null float64
Item_Weight                  11765 non-null float64
Outlet_Establishment_Year    14204 non-null int64
Outlet_Identifier            14204 non-null object
Outlet_Location_Type         14204 non-null object
Outlet_Size                  10188 non-null object
Outlet_Type                  14204 non-null object
dtypes: float64(4), int64(1), object(7)
memory usage: 1.4+ MB


In [96]:
df.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000
mean,141.004977,2181.288914,0.065953,12.792854,1997.830681
std,62.086938,1706.499616,0.051459,4.652502,8.371664
min,31.290000,33.290000,0.000000,4.555000,1985.000000
25%,94.012000,834.247400,0.027036,8.710000,1987.000000
50%,142.247000,1794.331000,0.054021,12.600000,1999.000000
75%,185.855600,3101.296400,0.094037,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


In [97]:
df.describe(include="all")

,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type
count,14204,14204,14204.000000,8523.000000,14204,14204.000000,11765.000000,14204.000000,14204,14204,10188,14204
unique,5,1559,NaN,NaN,16,NaN,NaN,NaN,10,3,3,4
top,Low Fat,FDT36,NaN,NaN,Fruits and Vegetables,NaN,NaN,NaN,OUT027,Tier 3,Medium,Supermarket Type1
freq,8485,10,NaN,NaN,2013,NaN,NaN,NaN,1559,5583,4655,9294
mean,NaN,NaN,141.004977,2181.288914,NaN,0.065953,12.792854,1997.830681,NaN,NaN,NaN,NaN
std,NaN,NaN,62.086938,1706.499616,NaN,0.051459,4.652502,8.371664,NaN,NaN,NaN,NaN
min,NaN,NaN,31.290000,33.290000,NaN,0.000000,4.555000,1985.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,94.012000,834.247400,NaN,0.027036,8.710000,1987.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,142.247000,1794.331000,NaN,0.054021,12.600000,1999.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,185.855600,3101.296400,NaN,0.094037,16.750000,2004.000000,NaN,NaN,NaN,NaN


In [98]:
df.duplicated().sum()

0

In [99]:
df.isna().sum()

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                  2439
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  4016
Outlet_Type                     0
dtype: int64

In [100]:
df["Item_Fat_Content"].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [101]:
df["Item_Fat_Content"].value_counts()

Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

In [102]:
df["Item_Fat_Content"].replace({"LF":"Low Fat","low fat":"Low Fat","reg":"Regular"},inplace=True)

In [103]:
df["Item_Fat_Content"].value_counts()

Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64

In [104]:
df["Item_Identifier_type"]=df["Item_Identifier"].apply(lambda x :x[:2])

In [105]:
df["Item_Visibility"].value_counts()

0.000000    879
0.076856      3
0.076841      3
0.077290      3
0.077169      3
0.076975      3
0.077011      3
0.076792      3
0.076483      3
0.046899      2
0.159844      2
0.135708      2
0.135944      2
0.072298      2
0.136008      2
0.136896      2
0.203401      2
0.081788      2
0.075049      2
0.058369      2
0.104784      2
0.080131      2
0.072411      2
0.024635      2
0.052069      2
0.079968      2
0.179192      2
0.079806      2
0.121767      2
0.080625      2
           ... 
0.013147      1
0.098790      1
0.073397      1
0.137756      1
0.017937      1
0.105126      1
0.044156      1
0.141639      1
0.094411      1
0.074883      1
0.039236      1
0.044497      1
0.051147      1
0.175061      1
0.039034      1
0.077508      1
0.078943      1
0.077170      1
0.042959      1
0.013173      1
0.069939      1
0.011305      1
0.069042      1
0.012216      1
0.064142      1
0.209684      1
0.019592      1
0.013530      1
0.008772      1
0.066817      1
Name: Item_Visibility, L

In [106]:
visibility= df["Item_Visibility"]

In [107]:
df.loc[df["Item_Visibility"]==0.000000,"Item_Visibility"] =np.NAN

In [108]:
df["Item_Visibility"].isna().sum()

879

In [109]:
#df.groupby("Item_Visibility").Item_weight.tranform

In [110]:
df["Item_Visibility"].fillna(0.000000,inplace=True)

In [111]:
df["Item_Weight"]=df.groupby("Item_Identifier").Item_Weight.transform(lambda x :x.fillna(x.mean()))

In [112]:
df["Item_Weight"].isna().sum()

0

In [113]:
df["Outlet_Establishment_Year"].unique()

array([1999, 2009, 1998, 1987, 1985, 2002, 2007, 1997, 2004], dtype=int64)

In [114]:
df["Outlet_Establishment_Year"]=2019-df["Outlet_Establishment_Year"]

In [115]:
df["Outlet_Identifier"].unique()

array(['OUT049', 'OUT018', 'OUT010', 'OUT013', 'OUT027', 'OUT045',
       'OUT017', 'OUT046', 'OUT035', 'OUT019'], dtype=object)

In [116]:
df["Outlet_Identifier_number"]=df["Outlet_Identifier"].apply(lambda x :x[4:]).astype("category")

In [117]:
df["Outlet_Size"].isna().sum()

4016

In [118]:
df["Outlet_Size"]=df.groupby("Outlet_Location_Type").Outlet_Size.transform(lambda x:x.fillna(x.mode()[0]))

In [119]:
del df["Outlet_Identifier_number"]
#addef for testing
df.columns

Index(['Item_Fat_Content', 'Item_Identifier', 'Item_MRP', 'Item_Outlet_Sales',
       'Item_Type', 'Item_Visibility', 'Item_Weight',
       'Outlet_Establishment_Year', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Size', 'Outlet_Type',
       'Item_Identifier_type'],
      dtype='object')

In [120]:
cat_column =df.select_dtypes(exclude=np.number).columns

In [121]:
cat_column

Index(['Item_Fat_Content', 'Item_Identifier', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Size', 'Outlet_Type',
       'Item_Identifier_type'],
      dtype='object')

In [122]:
num_column =df.select_dtypes(include=np.number).columns

In [123]:
num_column

Index(['Item_MRP', 'Item_Outlet_Sales', 'Item_Visibility', 'Item_Weight',
       'Outlet_Establishment_Year'],
      dtype='object')

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14204 entries, 0 to 5680
Data columns (total 13 columns):
Item_Fat_Content             14204 non-null object
Item_Identifier              14204 non-null object
Item_MRP                     14204 non-null float64
Item_Outlet_Sales            8523 non-null float64
Item_Type                    14204 non-null object
Item_Visibility              14204 non-null float64
Item_Weight                  14204 non-null float64
Outlet_Establishment_Year    14204 non-null int64
Outlet_Identifier            14204 non-null object
Outlet_Location_Type         14204 non-null object
Outlet_Size                  14204 non-null object
Outlet_Type                  14204 non-null object
Item_Identifier_type         14204 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 1.5+ MB


In [125]:
cat_column

Index(['Item_Fat_Content', 'Item_Identifier', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Size', 'Outlet_Type',
       'Item_Identifier_type'],
      dtype='object')

In [127]:
df_cat_dummy=pd.get_dummies(df[cat_column].drop(columns="Item_Identifier"))
#pd.get_dummies(df[cat_column])


In [131]:
 df_processed=pd.concat([df[num_column],df_cat_dummy,df["Item_Identifier"],df["Outlet_Identifier"]],axis="columns")

In [132]:
df_processed["Item_Outlet_Sales"].isna().sum()

5681

In [133]:
test_processed=df_processed[df_processed["Item_Outlet_Sales"].isna()]

In [136]:
train_processed=df_processed[df_processed["Item_Outlet_Sales"].notna()]

In [138]:
test_processed.shape

(5681, 48)

In [139]:
train_processed.shape

(8523, 48)

In [135]:
#Machine Leaning Model
y=train_processed["Item_Outlet_Sales"]

In [140]:
x=train_processed.drop(columns=["Item_Outlet_Sales","Item_Identifier","Outlet_Identifier"])

In [141]:
test_x=test_processed.drop(columns=["Item_Outlet_Sales","Item_Identifier","Outlet_Identifier"])

In [142]:
linear=LinearRegression()
linear.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [144]:
#RMSE
np.sqrt(np.mean(np.abs(cross_val_score(linear,x,y,cv=20,scoring="neg_mean_squared_error"))))

1131.694558970095

In [145]:
test_pred=linear.predict(test_x)

In [147]:
#just arrya
test_pred

array([1839.14326401, 1520.13694526, 1882.59095707, ..., 1822.74674364,
       3590.06296815, 1284.86003942])

In [148]:
test_pred=pd.DataFrame(test_pred,columns=["Item_Outlet_Sales"])

In [149]:
test_pred

,Item_Outlet_Sales
0,1839.143264
1,1520.136945
2,1882.590957
3,2577.602596
4,5129.053732
5,1952.476216
6,596.675544
7,2782.727237
8,1490.104023
9,3088.169269


In [155]:
linear_result=pd.concat([test_processed[["Item_Identifier","Outlet_Identifier"]],test_pred],axis=1)

In [156]:
linear_result.to_csv("linear_model.csv",index=False)